In [ ]:
''''''
import numpy as np
import scipy as sp
import scipy.signal
from matplotlib import pyplot as plt
from tqdm import tqdm
from cnld.api import *
from cnld import fem, bem, util, database

%matplotlib qt

geom = define.circular_cmut_1mhz_geometry()
geom.controldomain_nr = 1
# geom.radius = 32e-6
geom.controldomain_ntheta = 1
geom.damping_ratio1 = 0.1
geom.damping_ratio2 = 0.1
# print(geom)

layout = define.hexagonal_layout2(n=5, pitch=74e-6)
layout.geometries.append(geom)
layout.controldomains = define.generate_controldomainlist(layout)

grids = grid.generate_grids_from_layout(layout, refn=7)
# grids.bem.plot()

In [ ]:
dbsolver = solve.DatabaseSolver('circular_cmut_1mhz_hexagonal_5.db', freq_interp=4, overwrite=False)
dbsolver.layout = layout
dbsolver.grids = grids
dbsolver.freqs = 0, 30e6, 100e3

dbsolver.solve()

In [ ]:
freqs, ppfr = database.read_patch_to_patch_freq_resp('circular_cmut_1mhz_hexagonal_5.db')

fr = np.mean(np.sum(ppfr, axis=0), axis=0)
fr *= (2 * np.pi * freqs)**2

fig, ax = plt.subplots()
ax.plot(freqs / 1e6, 10 * np.log10(np.abs(fr) / np.abs(fr).max()), '.-')
ax.xlim(0, 30)

In [ ]:
fs = 40e6
_, vdc = define.linear_ramp(2e-6, 1 / fs)
vdc *= 30
pulse, _ = util.gausspulse(2e6, 0.25, fs=fs)
pulse *= 20

v = np.concatenate((vdc, 30 * np.ones(300), pulse + 30))
# v = np.concatenate((vdc, 30 * np.ones(400)))

transmit = define.Transmit()
transmit.waveforms.append(define.Waveform(voltage=v))
transmit.fs = fs

n = 1
vmax = np.max(v)
e_0 = 8.85418782e-12
pmax = e_0 / 2 * 50**2 / (geom.gap / geom.eps_r)**2 * 2
k = pmax / (10e-9)**n / 100
print(k)
lmda = 3 / 2 * k * 0.0

geom.contact_k = k
geom.contact_n = n
geom.contact_z0 = -geom.gap
geom.contact_lmda = lmda
layout.geometries = define.GeometryList([geom])

tsolver = solve.TimeSolver()
tsolver.layout = layout
tsolver.transmit = transmit
tsolver.dbfile = 'circular_cmut_1mhz_hexagonal_5.db'
tsolver.times = 0, 20e-6, 4e-9

# tsolver.recalculate_fir(interp=4)
tsolver.setup()
for i in tqdm(tsolver, total=len(tsolver)):
    pass

In [ ]:
fig, ax = plt.subplots()
ax.plot(np.mean(tsolver.displacement, axis=1))
ax.plot(tsolver.displacement[:,0])

fig, ax = plt.subplots()
ax.plot(tsolver.time, tsolver.voltage)

fs = 1 / (tsolver.time[1] - tsolver.time[0])
x = np.mean(tsolver.displacement, axis=1)
x = x[2000:4000]
x -= np.mean(x)
xdd = np.gradient(np.gradient(x))

win = sp.signal.tukey(len(xdd), 0.1)
freqs, esd = util.qesd(xdd, n=None, fs=fs, win=win)

fig, ax = plt.subplots()
ax.plot(freqs / 1e6, 10 * np.log10(esd / esd.max()))
ax.xlim(0, 50);